In [ ]:
# Progama Inventario Litoteca Nacional versión 1.0

pip install xlwt


import tkinter as tk
from tkinter import messagebox, StringVar, OptionMenu, Entry, ttk, Scrollbar
import psycopg2
import xlwt  # Para exportar a .xls

# 1. Función para conectar a la base de datos
def conectar_bd():
    try:
        conn = psycopg2.connect(
            dbname="inventario",
            user="postgres",
            password="postgres",  # Reemplaza con tu contraseña real
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa a la base de datos")
        return conn
    except Exception as e:
        print(f"Error de conexión: {e}")
        messagebox.showerror("Error de Conexión", f"No se pudo conectar a la base de datos: {e}")
        return None

# 2. Función para limpiar los campos del formulario
def limpiar_campos():
    centro_operacion_entry.delete(0, tk.END)
    secuencia_entry.delete(0, tk.END)
    codigo_barras_entry.delete(0, tk.END)
    serie_entry.delete(0, tk.END)
    placa_entry.delete(0, tk.END)
    descripcion_elemento_entry.delete(0, tk.END)
    tipo_parte_entry.delete(0, tk.END)
    tipo_almacenamiento_entry.delete(0, tk.END)
    fecha_ingreso_entry.delete(0, tk.END)
    costo_entry.delete(0, tk.END)
    cedula_resp_entry.delete(0, tk.END)
    responsable_inventario_entry.delete(0, tk.END)
    responsable_uso_var.set("")  # Limpiar el responsable de uso
    estado_var.set("")
    ubicacion_var.set("")  # Limpiar la ubicación
    secuencia_busqueda_entry.delete(0, tk.END)  # Limpiar búsqueda por secuencia

# 3. Función para cargar ubicaciones desde la base de datos
def cargar_ubicaciones():
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT ubicaciones_litoteca FROM public.ubicaciones_litoteca ORDER BY ubicaciones_litoteca")
            ubicaciones = cursor.fetchall()
            ubicaciones_list = [ubicacion[0] for ubicacion in ubicaciones if ubicacion[0] is not None]
            return ubicaciones_list
        except Exception as e:
            print(f"Error al cargar las ubicaciones: {e}")
            return []
        finally:
            cursor.close()
            conn.close()
    return []

# 4. Función para cargar los responsables de uso desde la base de datos
def cargar_responsables():
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("SELECT nombre FROM public.personal_litoteca ORDER BY nombre")
            responsables = cursor.fetchall()
            responsables_list = [responsable[0] for responsable in responsables if responsable[0] is not None]
            return responsables_list
        except Exception as e:
            print(f"Error al cargar los responsables: {e}")
            return []
        finally:
            cursor.close()
            conn.close()
    return []

# 5. Función para buscar un registro por Código de Barras y actualizar 'verificado' a TRUE automáticamente
def buscar_por_codigo_barras():
    codigo_barras = codigo_barras_busqueda_entry.get()
    if len(codigo_barras) < 6:
        return  # Si no hay suficientes caracteres, no realizar búsqueda

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                SELECT iv.centro_operacion, iv.secuencia, iv.codigo_barras, 
                       iv.serie, iv.placa, iv.descripcion_elemento, iv.tipo_parte, 
                       iv.tipo_almacenamiento, iv.fecha_ingreso, iv.costo, iv.cedula_resp,
                       iv.responsable_inventario, iv.responsable_uso, iv.estado_elemento, 
                       iv.ub_verificada_2024, iv.verificado
                FROM inventario_verificado iv
                WHERE iv.codigo_barras = %s;
            """, (codigo_barras,))
            resultado = cursor.fetchone()

            if resultado:
                limpiar_campos()  # Limpiar campos antes de cargar los nuevos datos
                centro_operacion_entry.insert(0, resultado[0] if resultado[0] else "N/A")
                secuencia_entry.insert(0, resultado[1] if resultado[1] else "N/A")
                codigo_barras_entry.insert(0, resultado[2] if resultado[2] else "N/A")
                serie_entry.insert(0, resultado[3] if resultado[3] else "N/A")
                placa_entry.insert(0, resultado[4] if resultado[4] else "N/A")
                descripcion_elemento_entry.insert(0, resultado[5] if resultado[5] else "N/A")
                tipo_parte_entry.insert(0, resultado[6] if resultado[6] else "N/A")
                tipo_almacenamiento_entry.insert(0, resultado[7] if resultado[7] else "N/A")
                fecha_ingreso_entry.insert(0, resultado[8] if resultado[8] else "N/A")
                costo_entry.insert(0, resultado[9] if resultado[9] else "N/A")
                cedula_resp_entry.insert(0, resultado[10] if resultado[10] else "N/A")
                responsable_inventario_entry.insert(0, resultado[11] if resultado[11] else "N/A")
                responsable_uso_var.set(resultado[12] if resultado[12] else "")
                estado_var.set(resultado[13] if resultado[13] else "N/A")
                ubicacion_var.set(resultado[14] if resultado[14] else "")

                # Copiar la secuencia automáticamente al campo de "Buscar por Secuencia"
                secuencia_busqueda_entry.insert(0, resultado[1] if resultado[1] else "")

                # Actualizar automáticamente el campo 'verificado' a TRUE en la base de datos
                cursor.execute("""
                    UPDATE inventario_verificado
                    SET verificado = TRUE
                    WHERE codigo_barras = %s;
                """, (codigo_barras,))
                conn.commit()

                # Registrar búsqueda en la tabla de resultados
                registrar_busqueda(resultado[2], resultado[1], resultado[3], resultado[4], resultado[5], resultado[13], resultado[14], resultado[12])

                # Limpiar solo el campo de búsqueda del código de barras
                codigo_barras_busqueda_entry.delete(0, tk.END)

            else:
                limpiar_campos()  # Limpiar campos si no encuentra el código de barras
                messagebox.showinfo("Información", "No se encontró ningún registro con ese Código de Barras.")
        except Exception as e:
            print(f"Error al ejecutar la consulta: {e}")
        finally:
            cursor.close()
            conn.close()

# 6. Función para registrar las búsquedas en la tabla
def registrar_busqueda(codigo_barras, secuencia, serie, placa, descripcion, estado, ubicacion, responsable):
    # Verificar si se ha seleccionado una ubicación
    if not ubicacion_var.get():
        messagebox.showwarning("Advertencia", "Seleccione la ubicación del elemento")
        return

    # Añadir los resultados de la búsqueda a la tabla
    tree.insert('', 'end', values=(codigo_barras, secuencia, serie, placa, descripcion, estado, ubicacion, responsable))

# 7. Función para exportar los resultados de búsqueda a un archivo .xls
def exportar_consulta():
    archivo = 'consulta_sesion.xls'
    wb = xlwt.Workbook()
    ws = wb.add_sheet('Consultas')

    # Escribir la cabecera del Excel
    cabecera = ["Código de Barras", "Secuencia", "Serie", "Placa", "Descripción", "Estado", "Ubicación", "Responsable de Uso"]
    for col_num, header in enumerate(cabecera):
        ws.write(0, col_num, header)

    # Escribir los resultados de búsqueda en el archivo Excel
    for row_num, row_id in enumerate(tree.get_children(), start=1):
        row = tree.item(row_id)['values']
        for col_num, value in enumerate(row):
            ws.write(row_num, col_num, value)

    wb.save(archivo)
    messagebox.showinfo("Exportar", f"Consulta exportada a {archivo} exitosamente.")

# 8. Función para insertar un nuevo registro manualmente (Limpiar formulario y generar un código de barras nuevo)
def insertar_nuevo_registro():
    limpiar_campos()  # Limpiar campos para ingresar un nuevo registro
    
    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            # Generar un nuevo código de barras automáticamente
            cursor.execute("SELECT MAX(codigo_barras) FROM inventario_verificado;")
            max_codigo_barras = cursor.fetchone()[0]
            nuevo_codigo_barras = str(int(max_codigo_barras) + 1) if max_codigo_barras else "000001"

            # Insertar un nuevo registro vacío
            cursor.execute("""
                INSERT INTO inventario_verificado (centro_operacion, secuencia, codigo_barras, serie, placa, descripcion_elemento, tipo_parte, tipo_almacenamiento, fecha_ingreso, costo, cedula_resp, responsable_inventario, responsable_uso, estado_elemento, ub_verificada_2024, verificado)
                VALUES ('', '', %s, '', '', '', '', '', NULL, '', '', '', '', '', '', FALSE);
            """, (nuevo_codigo_barras,))
            conn.commit()

            # Cargar el nuevo código de barras en el campo correspondiente
            codigo_barras_entry.insert(0, nuevo_codigo_barras)
            messagebox.showinfo("Nuevo Registro", "Se ha insertado un nuevo registro en blanco.")
        except Exception as e:
            conn.rollback()
            print(f"Error al insertar el nuevo registro: {e}")
        finally:
            cursor.close()
            conn.close()

# 9. Función para buscar un registro por Secuencia y mostrar resultados en el Frame Derecho
def buscar_por_secuencia():
    secuencia = secuencia_busqueda_entry.get()
    if not secuencia:
        messagebox.showwarning("Advertencia", "Por favor, ingrese una Secuencia.")
        return

    conn = conectar_bd()
    if conn:
        cursor = conn.cursor()
        try:
            cursor.execute("""
                SELECT iv.secuencia, iv.codigo_barras, iv.serie, iv.placa, iv.descripcion_elemento, 
                       iv.estado_elemento, iv.ub_verificada_2024, iv.responsable_uso
                FROM inventario_verificado iv
                WHERE iv.secuencia = %s;
            """, (secuencia,))
            resultado = cursor.fetchall()

            if resultado:
                # Limpiar el frame de resultados antes de mostrar los nuevos datos
                for widget in result_frame.winfo_children():
                    widget.destroy()

                for res in resultado:
                    result_text = f"Secuencia: {res[0]}\nCódigo de Barras: {res[1]}\nSerie: {res[2]}\nPlaca: {res[3]}\nDescripción: {res[4]}\nEstado: {res[5]}\nUbicación: {res[6]}\nResponsable: {res[7]}\n" + "-"*30
                    tk.Label(result_frame, text=result_text, justify="left", anchor="w").pack(fill="both")
            else:
                # Limpiar el frame de resultados si no encuentra la secuencia
                for widget in result_frame.winfo_children():
                    widget.destroy()
                tk.Label(result_frame, text="No se encontró ningún registro con esa Secuencia.").pack()

        except Exception as e:
            print(f"Error al buscar por secuencia: {e}")
        finally:
            cursor.close()
            conn.close()

# 10. Configuración de la ventana principal de Tkinter
root = tk.Tk()
root.title("Inventario Litoteca Nacional")

# Crear los frames para dividir la pantalla
frame_left = tk.Frame(root)
frame_left.grid(row=0, column=0, padx=20, pady=20, sticky="n")

frame_right = tk.Frame(root)
frame_right.grid(row=0, column=1, padx=20, pady=20, sticky="n")

# Búsqueda por código de barras
tk.Label(frame_left, text="Buscar por Código de Barras:").grid(row=0, column=0, sticky="w")
codigo_barras_busqueda_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
codigo_barras_busqueda_entry.grid(row=0, column=1, padx=5)
codigo_barras_busqueda_entry.bind("<KeyRelease>", lambda event: buscar_por_codigo_barras())  # Búsqueda automática tras 6 caracteres

tk.Button(frame_left, text="Buscar", command=buscar_por_codigo_barras).grid(row=0, column=2, padx=5)

# Campos del formulario
centro_operacion_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
secuencia_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
codigo_barras_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
serie_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
placa_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
descripcion_elemento_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
tipo_parte_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
tipo_almacenamiento_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
fecha_ingreso_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
costo_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
cedula_resp_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%
responsable_inventario_entry = tk.Entry(frame_left, width=30)  # Ancho incrementado en un 25%

# Variables para Combobox
ubicacion_var = StringVar()
responsable_uso_var = StringVar()

# Agregamos las etiquetas y campos de entrada de manera explícita
campos = [
    ("Centro Operación", centro_operacion_entry),
    ("Secuencia", secuencia_entry),
    ("Código de Barras", codigo_barras_entry),
    ("Serie", serie_entry),
    ("Placa", placa_entry),
    ("Descripción Elemento", descripcion_elemento_entry),
    ("Tipo Parte", tipo_parte_entry),
    ("Tipo Almacenamiento", tipo_almacenamiento_entry),
    ("Fecha Ingreso", fecha_ingreso_entry),
    ("Costo", costo_entry),
    ("Cédula Responsable", cedula_resp_entry),
    ("Responsable Inventario", responsable_inventario_entry)
]

for i, (label_text, entry) in enumerate(campos):
    tk.Label(frame_left, text=f"{label_text}:").grid(row=i + 1, column=0, sticky="w")
    entry.grid(row=i + 1, column=1, columnspan=2, padx=5, pady=2)

# Menú desplegable para estado
tk.Label(frame_left, text="Estado Elemento:").grid(row=len(campos) + 1, column=0, sticky="w")
estado_var = StringVar(frame_left)
estado_menu = OptionMenu(frame_left, estado_var, "Bueno", "Inservible")
estado_menu.grid(row=len(campos) + 1, column=1, columnspan=2, padx=5)

# Menú desplegable para el campo "Ubicación Litoteca"
tk.Label(frame_left, text="Ubicación Litoteca:").grid(row=len(campos) + 2, column=0, sticky="w")
ubicacion_menu = ttk.Combobox(frame_left, textvariable=ubicacion_var, values=cargar_ubicaciones(), width=30)  # Incremento en 25%
ubicacion_menu.grid(row=len(campos) + 2, column=1, columnspan=2, padx=5)
ubicacion_menu.bind("<KeyRelease>", lambda event: actualizar_ubicacion(event))

# Menú desplegable para el campo "Responsable de Uso"
tk.Label(frame_left, text="Responsable de Uso:").grid(row=len(campos) + 3, column=0, sticky="w")
responsable_uso_menu = ttk.Combobox(frame_left, textvariable=responsable_uso_var, values=cargar_responsables(), width=30)  # Incremento en 25%
responsable_uso_menu.grid(row=len(campos) + 3, column=1, columnspan=2, padx=5)
responsable_uso_menu.bind("<KeyRelease>", lambda event: actualizar_responsable_uso(event))

# Botones de control
tk.Button(frame_left, text="Nuevo Registro", command=lambda: [insertar_nuevo_registro(), cargar_responsable()]).grid(row=len(campos) + 4, column=0, padx=5, pady=10)
tk.Button(frame_left, text="Limpiar Campos", command=limpiar_campos).grid(row=len(campos) + 4, column=1, padx=5, pady=10)
tk.Button(frame_left, text="Guardar Cambios", command=guardar_cambios).grid(row=len(campos) + 4, column=2, padx=5, pady=10)

# Botón para exportar los resultados de búsqueda
tk.Button(frame_left, text="Exportar Consulta", command=exportar_consulta).grid(row=len(campos) + 5, column=1, padx=5, pady=10)

# ------------------ Frame Derecho: Búsqueda por Secuencia ------------------
tk.Label(frame_right, text="Buscar por Secuencia:").grid(row=0, column=0, sticky="w")
secuencia_busqueda_entry = tk.Entry(frame_right)
secuencia_busqueda_entry.grid(row=0, column=1, padx=5)
tk.Button(frame_right, text="Buscar", command=buscar_por_secuencia).grid(row=0, column=2, padx=5)

# Frame para los resultados de búsqueda por secuencia
result_frame = tk.Frame(frame_right)
result_frame.grid(row=1, column=0, columnspan=3, pady=10)

# ---------------- Tabla para registrar las búsquedas realizadas con Scrollbar ----------------
scrollbar = Scrollbar(root)
scrollbar.grid(row=1, column=2, sticky='ns')

columns = ("Código de Barras", "Secuencia", "Serie", "Placa", "Descripción", "Estado", "Ubicación", "Responsable de Uso")
tree = ttk.Treeview(root, columns=columns, show="headings", yscrollcommand=scrollbar.set)
scrollbar.config(command=tree.yview)

tree.heading("Código de Barras", text="Código de Barras")
tree.heading("Secuencia", text="Secuencia")
tree.heading("Serie", text="Serie")
tree.heading("Placa", text="Placa")
tree.heading("Descripción", text="Descripción")
tree.heading("Estado", text="Estado")
tree.heading("Ubicación", text="Ubicación")
tree.heading("Responsable de Uso", text="Responsable de Uso")
tree.grid(row=1, column=0, columnspan=2, padx=20, pady=20)

# Copyright y finalización
copyright_label = tk.Label(root, text="© 2024 Jorge A Melo", font=("Arial", 10))
copyright_label.grid(row=2, column=0, columnspan=2, pady=10)

# Cargar ubicaciones al iniciar la aplicación
ubicaciones_list = cargar_ubicaciones()
responsables_list = cargar_responsables()

# Ejecutar la aplicación
root.mainloop()